In [2]:
#LOAD DATA
#Load
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import sqlite3
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import requests     # to get images
import shutil       # to save files locally
import datetime
from scipy.stats import norm
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import os
from pathlib import Path
# NBA API
#from nba_api.stats.static import players
#from nba_api.stats.endpoints import commonplayerinfo
#from nba_api.stats.static import teams

In [3]:
minutes_df = pd.read_excel('data_outputs_whoplayed_3/0_all_attempt_21.xlsx')
print(f' The shape of the DF: {minutes_df.shape}')
minutes_df.head(3)

 The shape of the DF: (5741, 1979)


,Unnamed: 0,Matchup_GameDate,MIL_Thanasis Antetokounmpo_min,MIL_Jrue Holiday_min,MIL_Brook Lopez_min,MIL_Grayson Allen_min,MIL_Jordan Nwora_min,MIL_Pat Connaughton_min,MIL_Khris Middleton_min,MIL_George Hill_min,...,HOU_Isaiah Canaan_min,NYK_Mindaugas Kuzminskas_min,MIL_Joel Bolomboy_min,BOS_Jarell Eddie_min,HOU_Brandan Wright_min,TOR_Nigel Hayes_min,MEM_Marquis Teague_min,POR_Georgios Papagiannis_min,HOU_Chinanu Onuaku_min,HOU_Aaron Jackson_min
0,0,ATL @ BKN_01_01_2021,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,ATL @ BKN_01_09_2019,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,ATL @ BKN_01_12_2020,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
season_21 = pd.read_excel('data_outputs/player_traditional_box_scores_2021_22.xlsx')
season_20 = pd.read_excel('data_outputs/player_traditional_box_scores_2020_21.xlsx')
season_19 = pd.read_excel('data_outputs/player_traditional_box_scores_2019_20.xlsx')
season_18 = pd.read_excel('data_outputs/player_traditional_box_scores_2018_19.xlsx')
season_17 = pd.read_excel('data_outputs/player_traditional_box_scores_2017_18.xlsx')

In [4]:
seasons = [season_21, season_20, season_19, season_18, season_17]
all_seasons = []
szn = 2021

for season in seasons:
    season['Season'] = szn
    all_seasons.append(season)
    szn -= 1

all_seasons = pd.concat(all_seasons)
all_seasons

,Unnamed: 0,Unnamed: 0.1,Player,Team,Match Up,Game Date,Season,W/L,MIN,PTS,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-,team_date_id
0,1,1,Thanasis Antetokounmpo,MIL,MIL vs. BKN,10/19/2021,2021,W,12.0,6.0,...,1.0,3.0,4.0,1.0,0.0,1.0,0.0,1.0,8.0,MIL_10/19/2021
1,2,2,Blake Griffin,BKN,BKN @ MIL,10/19/2021,2021,L,23.0,6.0,...,1.0,4.0,5.0,0.0,1.0,0.0,1.0,1.0,-6.0,BKN_10/19/2021
2,3,3,Joe Harris,BKN,BKN @ MIL,10/19/2021,2021,L,31.0,9.0,...,0.0,2.0,2.0,2.0,0.0,0.0,2.0,1.0,0.0,BKN_10/19/2021
3,4,4,Jrue Holiday,MIL,MIL vs. BKN,10/19/2021,2021,W,18.0,12.0,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,17.0,MIL_10/19/2021
4,5,5,Nic Claxton,BKN,BKN @ MIL,10/19/2021,2021,L,24.0,12.0,...,0.0,7.0,7.0,0.0,0.0,0.0,0.0,3.0,-15.0,BKN_10/19/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26108,1869,1869,Gary Harris,DEN,DEN @ MIN,04/11/2018,2017,L,21.0,10.0,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,-18.0,NaN
26109,1870,1870,Wilson Chandler,DEN,DEN @ MIN,04/11/2018,2017,L,48.0,0.0,...,1.0,8.0,9.0,2.0,1.0,0.0,3.0,5.0,1.0,NaN
26110,1871,1871,Taj Gibson,MIN,MIN vs. DEN,04/11/2018,2017,W,35.0,8.0,...,0.0,6.0,6.0,0.0,2.0,1.0,1.0,5.0,-4.0,NaN
26111,1872,1872,Paul Millsap,DEN,DEN @ MIN,04/11/2018,2017,L,36.0,10.0,...,1.0,6.0,7.0,1.0,0.0,1.0,1.0,4.0,0.0,NaN


In [5]:
# Part 1
cols = []
data = all_seasons
data['team_player'] = data['Team'] + '_' + data['Player']
home_games = data.loc[data['Match\xa0Up'].astype(str).str.contains('vs')]
away_games = data.loc[data['Match\xa0Up'].astype(str).str.contains('@')]

# Fix so all games have the same format
home_games['Match\xa0Up'] = home_games['Match\xa0Up'].astype(str).str[-3:] + ' @ ' + home_games['Match\xa0Up'].astype(str).str[0:3]
print(f' The shape of the DF: {home_games.shape}')
home_games.head(2)

 The shape of the DF: (60966, 30)


<ipython-input-5-04d5491b8a04>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_games['Match\xa0Up'] = home_games['Match\xa0Up'].astype(str).str[-3:] + ' @ ' + home_games['Match\xa0Up'].astype(str).str[0:3]


,Unnamed: 0,Unnamed: 0.1,Player,Team,Match Up,Game Date,Season,W/L,MIN,PTS,...,DREB,REB,AST,STL,BLK,TOV,PF,+/-,team_date_id,team_player
0,1,1,Thanasis Antetokounmpo,MIL,BKN @ MIL,10/19/2021,2021,W,12.0,6.0,...,3.0,4.0,1.0,0.0,1.0,0.0,1.0,8.0,MIL_10/19/2021,MIL_Thanasis Antetokounmpo
3,4,4,Jrue Holiday,MIL,BKN @ MIL,10/19/2021,2021,W,18.0,12.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,17.0,MIL_10/19/2021,MIL_Jrue Holiday


In [6]:
data2 = pd.concat([home_games, away_games])
data2.head(2)

,Unnamed: 0,Unnamed: 0.1,Player,Team,Match Up,Game Date,Season,W/L,MIN,PTS,...,DREB,REB,AST,STL,BLK,TOV,PF,+/-,team_date_id,team_player
0,1,1,Thanasis Antetokounmpo,MIL,BKN @ MIL,10/19/2021,2021,W,12.0,6.0,...,3.0,4.0,1.0,0.0,1.0,0.0,1.0,8.0,MIL_10/19/2021,MIL_Thanasis Antetokounmpo
3,4,4,Jrue Holiday,MIL,BKN @ MIL,10/19/2021,2021,W,18.0,12.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,17.0,MIL_10/19/2021,MIL_Jrue Holiday


In [7]:
# Add matchup_gamedate to data
data2['Matchup_GameDate'] = data2['Match\xa0Up'].astype(str) + '_' + data2['Game\xa0Date'].astype(str).str.replace('/', '_')
data2.head(2)

,Unnamed: 0,Unnamed: 0.1,Player,Team,Match Up,Game Date,Season,W/L,MIN,PTS,...,REB,AST,STL,BLK,TOV,PF,+/-,team_date_id,team_player,Matchup_GameDate
0,1,1,Thanasis Antetokounmpo,MIL,BKN @ MIL,10/19/2021,2021,W,12.0,6.0,...,4.0,1.0,0.0,1.0,0.0,1.0,8.0,MIL_10/19/2021,MIL_Thanasis Antetokounmpo,BKN @ MIL_10_19_2021
3,4,4,Jrue Holiday,MIL,BKN @ MIL,10/19/2021,2021,W,18.0,12.0,...,1.0,1.0,1.0,0.0,0.0,0.0,17.0,MIL_10/19/2021,MIL_Jrue Holiday,BKN @ MIL_10_19_2021


In [19]:
# Part 2
player_team_namez = data2['team_player'].unique()        # Get unique player names
for player_team in player_team_namez:                       # Add players as columns
    data2[str(player_team) + '_played'] = np.where( data2['team_player'] == str(player_team), 1, 0)
    data2[str(player_team) + '_min'] = np.where( data2['team_player'] == str(player_team), data2['MIN'], 0)
    cols2 = str(player_team) + '_min'
    cols.append(cols2)
grouped = data2.groupby('Matchup_GameDate')[cols].sum()
grouped.to_excel('data_outputs_whoplayed_3\\0_all_attempt_21.xlsx')

grouped

,MIL_Thanasis Antetokounmpo_min,MIL_Jrue Holiday_min,MIL_Brook Lopez_min,MIL_Grayson Allen_min,MIL_Jordan Nwora_min,MIL_Pat Connaughton_min,MIL_Khris Middleton_min,MIL_George Hill_min,MIL_Justin Robinson_min,MIL_Sandro Mamukelashvili_min,...,HOU_Isaiah Canaan_min,NYK_Mindaugas Kuzminskas_min,MIL_Joel Bolomboy_min,BOS_Jarell Eddie_min,HOU_Brandan Wright_min,TOR_Nigel Hayes_min,MEM_Marquis Teague_min,POR_Georgios Papagiannis_min,HOU_Chinanu Onuaku_min,HOU_Aaron Jackson_min
Matchup_GameDate,,,,,,,,,,,,,,,,,,,,,
ATL @ BKN_01_01_2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ATL @ BKN_01_09_2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ATL @ BKN_01_12_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ATL @ BKN_10_22_2017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ATL @ BKN_11_03_2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WAS @ UTA_02_28_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WAS @ UTA_03_29_2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WAS @ UTA_04_12_2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
include = ['Player',
 'Team',
 'Match\xa0Up',
 'Game\xa0Date',
 'Season','MIN', 'W/L', 'team_player', 'Matchup_GameDate']

check = data2[include]
check

,Player,Team,Match Up,Game Date,Season,MIN,W/L,team_player,Matchup_GameDate
0,Thanasis Antetokounmpo,MIL,BKN @ MIL,10/19/2021,2021,12.0,W,MIL_Thanasis Antetokounmpo,BKN @ MIL_10_19_2021
3,Jrue Holiday,MIL,BKN @ MIL,10/19/2021,2021,18.0,W,MIL_Jrue Holiday,BKN @ MIL_10_19_2021
5,Brook Lopez,MIL,BKN @ MIL,10/19/2021,2021,28.0,W,MIL_Brook Lopez,BKN @ MIL_10_19_2021
6,Grayson Allen,MIL,BKN @ MIL,10/19/2021,2021,28.0,W,MIL_Grayson Allen,BKN @ MIL_10_19_2021
7,Jordan Nwora,MIL,BKN @ MIL,10/19/2021,2021,26.0,W,MIL_Jordan Nwora,BKN @ MIL_10_19_2021
...,...,...,...,...,...,...,...,...,...
26101,Jamal Murray,DEN,DEN @ MIN,04/11/2018,2017,43.0,L,DEN_Jamal Murray,DEN @ MIN_04_11_2018
26107,Mason Plumlee,DEN,DEN @ MIN,04/11/2018,2017,12.0,L,DEN_Mason Plumlee,DEN @ MIN_04_11_2018
26108,Gary Harris,DEN,DEN @ MIN,04/11/2018,2017,21.0,L,DEN_Gary Harris,DEN @ MIN_04_11_2018
26109,Wilson Chandler,DEN,DEN @ MIN,04/11/2018,2017,48.0,L,DEN_Wilson Chandler,DEN @ MIN_04_11_2018


In [22]:
grouped = grouped.reset_index()
grouped = grouped.loc[:,~grouped.columns.duplicated()].copy()

In [25]:
grouped.to_excel('data_outputs_whoplayed_3\\0_all_attempt_21.xlsx')

In [23]:
one_game = grouped.loc[grouped['Matchup_GameDate'] == 'ATL @ BKN_01_01_2021']
one_game = one_game.T
one_game

,0
Matchup_GameDate,ATL @ BKN_01_01_2021
MIL_Thanasis Antetokounmpo_min,0
MIL_Jrue Holiday_min,0
MIL_Brook Lopez_min,0
MIL_Grayson Allen_min,0
...,...
TOR_Nigel Hayes_min,0
MEM_Marquis Teague_min,0
POR_Georgios Papagiannis_min,0
HOU_Chinanu Onuaku_min,0


In [24]:
one_game.loc[one_game[0] != 0]


,0
Matchup_GameDate,ATL @ BKN_01_01_2021
ATL_John Collins_min,25
ATL_Trae Young_min,32
ATL_Cam Reddish_min,31
ATL_Clint Capela_min,30
ATL_Bogdan Bogdanovic_min,26
ATL_De'Andre Hunter_min,31
ATL_Skylar Mays_min,1
ATL_Solomon Hill_min,24
ATL_Kevin Huerter_min,21


## Add Minutes Data to Win Data

![](xmind_data.png)


In [26]:
minutes_df = grouped

In [29]:
minutes_df.head(3)

,Matchup_GameDate,MIL_Thanasis Antetokounmpo_min,MIL_Jrue Holiday_min,MIL_Brook Lopez_min,MIL_Grayson Allen_min,MIL_Jordan Nwora_min,MIL_Pat Connaughton_min,MIL_Khris Middleton_min,MIL_George Hill_min,MIL_Justin Robinson_min,...,HOU_Isaiah Canaan_min,NYK_Mindaugas Kuzminskas_min,MIL_Joel Bolomboy_min,BOS_Jarell Eddie_min,HOU_Brandan Wright_min,TOR_Nigel Hayes_min,MEM_Marquis Teague_min,POR_Georgios Papagiannis_min,HOU_Chinanu Onuaku_min,HOU_Aaron Jackson_min
0,ATL @ BKN_01_01_2021,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ATL @ BKN_01_09_2019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ATL @ BKN_01_12_2020,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
game_df = pd.read_excel('Game_by_Game_Win_data_17-21.xlsx')

In [28]:
game_df.head(3)

,Unnamed: 0,Unnamed: 0_x,Team,Match Up,Home_team_x,Away_team_x,GameDate,Hm_Date_id,W/L,MIN,...,h1,h2,h3,h4,h5,Home_team_y,Away_team_y,hm_date_id,season,win?
0,0,1,CLE,CLE vs. MIL,CLE,MIL,04/10/2022,CLE-04-10-2022,W,48,...,Evan Mobley,Darius Garland,Lauri Markkanen,Caris LeVert,Isaac Okoro,CLE,MIL,CLE-04-10-2022,2021,1
1,1,2,LAL,LAL @ DEN,DEN,LAL,04/10/2022,DEN-04-10-2022,W,53,...,Jeff Green,Austin Rivers,Bryn Forbes,Bones Hyland,DeMarcus Cousins,DEN,LAL,DEN-04-10-2022,2021,1
2,2,3,UTA,UTA @ POR,POR,UTA,04/10/2022,POR-04-10-2022,W,48,...,Reggie Perry,Didi Louzada,Brandon Williams,Keon Johnson,CJ Elleby,POR,UTA,POR-04-10-2022,2021,1


In [30]:
home_games2 = game_df.loc[game_df['Match\xa0Up'].astype(str).str.contains('vs')]
away_games2 = game_df.loc[game_df['Match\xa0Up'].astype(str).str.contains('@')]

# Fix so all games have the same format
home_games2['Match\xa0Up'] = home_games2['Match\xa0Up'].astype(str).str[-3:] + ' @ ' + home_games2['Match\xa0Up'].astype(str).str[0:3]
print(f' The shape of the DF: {home_games2.shape}')
home_games2.head(2)

 The shape of the DF: (5812, 74)


<ipython-input-30-a627cf5a18de>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  home_games2['Match\xa0Up'] = home_games2['Match\xa0Up'].astype(str).str[-3:] + ' @ ' + home_games2['Match\xa0Up'].astype(str).str[0:3]


,Unnamed: 0,Unnamed: 0_x,Team,Match Up,Home_team_x,Away_team_x,GameDate,Hm_Date_id,W/L,MIN,...,h1,h2,h3,h4,h5,Home_team_y,Away_team_y,hm_date_id,season,win?
0,0,1,CLE,MIL @ CLE,CLE,MIL,04/10/2022,CLE-04-10-2022,W,48,...,Evan Mobley,Darius Garland,Lauri Markkanen,Caris LeVert,Isaac Okoro,CLE,MIL,CLE-04-10-2022,2021,1
4,4,5,DAL,SAS @ DAL,DAL,SAS,04/10/2022,DAL-04-10-2022,W,48,...,Dwight Powell,Luka Doncic,Jalen Brunson,Reggie Bullock,Dorian Finney-Smith,DAL,SAS,DAL-04-10-2022,2021,1


In [31]:
data3 = pd.concat([home_games2, away_games2])
data3.head(2)

,Unnamed: 0,Unnamed: 0_x,Team,Match Up,Home_team_x,Away_team_x,GameDate,Hm_Date_id,W/L,MIN,...,h1,h2,h3,h4,h5,Home_team_y,Away_team_y,hm_date_id,season,win?
0,0,1,CLE,MIL @ CLE,CLE,MIL,04/10/2022,CLE-04-10-2022,W,48,...,Evan Mobley,Darius Garland,Lauri Markkanen,Caris LeVert,Isaac Okoro,CLE,MIL,CLE-04-10-2022,2021,1
4,4,5,DAL,SAS @ DAL,DAL,SAS,04/10/2022,DAL-04-10-2022,W,48,...,Dwight Powell,Luka Doncic,Jalen Brunson,Reggie Bullock,Dorian Finney-Smith,DAL,SAS,DAL-04-10-2022,2021,1


In [33]:
# Add matchup_gamedate to data
data3['Matchup_GameDate'] = data3['Match\xa0Up'].astype(str) + '_' + data3['GameDate'].astype(str).str.replace('/', '_')
data3.head(2)

,Unnamed: 0,Unnamed: 0_x,Team,Match Up,Home_team_x,Away_team_x,GameDate,Hm_Date_id,W/L,MIN,...,h2,h3,h4,h5,Home_team_y,Away_team_y,hm_date_id,season,win?,Matchup_GameDate
0,0,1,CLE,MIL @ CLE,CLE,MIL,04/10/2022,CLE-04-10-2022,W,48,...,Darius Garland,Lauri Markkanen,Caris LeVert,Isaac Okoro,CLE,MIL,CLE-04-10-2022,2021,1,MIL @ CLE_04_10_2022
4,4,5,DAL,SAS @ DAL,DAL,SAS,04/10/2022,DAL-04-10-2022,W,48,...,Luka Doncic,Jalen Brunson,Reggie Bullock,Dorian Finney-Smith,DAL,SAS,DAL-04-10-2022,2021,1,SAS @ DAL_04_10_2022


In [34]:
data3.to_excel('Game_by_Game_Win_data_17-21.xlsx')

In [8]:
data3 = pd.read_excel('Game_by_Game_Win_data_17-21.xlsx')

In [37]:
new_df = pd.merge(data3, minutes_df, on='Matchup_GameDate', how = 'left')
print(f' The shape of the DF: {new_df.shape}')
new_df.head(4)

 The shape of the DF: (11623, 2052)


,Unnamed: 0,Unnamed: 0_x,Team,Match Up,Home_team_x,Away_team_x,GameDate,Hm_Date_id,W/L,MIN,...,HOU_Isaiah Canaan_min,NYK_Mindaugas Kuzminskas_min,MIL_Joel Bolomboy_min,BOS_Jarell Eddie_min,HOU_Brandan Wright_min,TOR_Nigel Hayes_min,MEM_Marquis Teague_min,POR_Georgios Papagiannis_min,HOU_Chinanu Onuaku_min,HOU_Aaron Jackson_min
0,0,1,CLE,MIL @ CLE,CLE,MIL,04/10/2022,CLE-04-10-2022,W,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,5,DAL,SAS @ DAL,DAL,SAS,04/10/2022,DAL-04-10-2022,W,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,7,POR,UTA @ POR,POR,UTA,04/10/2022,POR-04-10-2022,L,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,8,HOU,ATL @ HOU,HOU,ATL,04/10/2022,HOU-04-10-2022,L,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
player_df = pd.read_excel('All_Player_Data_asofJuly8_deciles.xlsx')

In [6]:
player_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Player,Team,Age,OFFRTG,DEFRTG,NETRTG,AST%,...,height_decile,wingspan_decile,2ptContested%_decile,3pt_CatchShoot_decile,3pt_Defense%_decile,DFG%_decile,Reb%_decile,AstRatio_decile,TovRatio_decile,season_position
0,0,0,0,Dwight Powell,DAL,30,113.2,110.6,2.5,8.1,...,8.0,3.0,9.0,7.0,4.0,4.0,3.0,7.0,6.0,2021_PF
1,5,5,5,Chris Boucher,TOR,29,111.0,105.7,5.3,2.2,...,3.0,8.0,7.0,2.0,1.0,0.0,7.0,0.0,0.0,2021_PF
2,15,15,15,Grant Williams,BOS,23,112.5,108.0,4.5,5.8,...,0.0,0.0,4.0,9.0,7.0,7.0,0.0,4.0,6.0,2021_PF
3,18,18,18,Naz Reid,MIN,22,105.5,107.9,-2.4,9.1,...,6.0,7.0,4.0,6.0,8.0,0.0,3.0,2.0,8.0,2021_PF
4,20,20,20,Georges Niang,PHI,28,110.9,109.0,1.9,9.0,...,1.0,0.0,5.0,9.0,9.0,9.0,0.0,5.0,2.0,2021_PF


In [4]:
decile_options = pd.read_excel('decile_weight_options.xlsx')
decile_options.head(5)

,Unnamed: 0,index,height,wingspan,Contested2pt,CatchShoot3pt,Defense3pt,def_fg,reb,ast,tov,total
0,0,739788,0.01,0.03,0.15,0.15,0.15,0.15,0.12,0.12,0.12,1
1,1,1460747,0.01,0.07,0.13,0.13,0.15,0.15,0.12,0.12,0.12,1
2,2,1465247,0.01,0.07,0.13,0.15,0.13,0.15,0.12,0.12,0.12,1
3,3,1465872,0.01,0.07,0.13,0.15,0.15,0.13,0.12,0.12,0.12,1
4,4,1496747,0.01,0.07,0.15,0.13,0.13,0.15,0.12,0.12,0.12,1


In [42]:
decile_options = decile_options.drop(columns = ['index'])
decile_options.head()

,Unnamed: 0,height,wingspan,Contested2pt,CatchShoot3pt,Defense3pt,def_fg,reb,ast,tov,total
0,0,0.01,0.03,0.15,0.15,0.15,0.15,0.12,0.12,0.12,1
1,1,0.01,0.07,0.13,0.13,0.15,0.15,0.12,0.12,0.12,1
2,2,0.01,0.07,0.13,0.15,0.13,0.15,0.12,0.12,0.12,1
3,3,0.01,0.07,0.13,0.15,0.15,0.13,0.12,0.12,0.12,1
4,4,0.01,0.07,0.15,0.13,0.13,0.15,0.12,0.12,0.12,1


In [48]:


player_df['decile_option 1 Total'] =(
                                    (player_df['height_decile'] * decile_options['height'].values[0]) + 
                                    (player_df['wingspan_decile'] * decile_options['wingspan'].values[0]) +
                                    (player_df['2ptContested%_decile'] * decile_options['Contested2pt'].values[0]) +
                                    (player_df['3pt_CatchShoot_decile'] * decile_options['CatchShoot3pt'].values[0]) -
                                    (player_df['3pt_Defense%_decile'] * decile_options['Defense3pt'].values[0]) -
                                    (player_df['DFG%_decile'] * decile_options['def_fg'].values[0]) +
                                    (player_df['Reb%_decile'] * decile_options['reb'].values[0]) +
                                    (player_df['AstRatio_decile'] * decile_options['ast'].values[0]) -
                                    (player_df['TovRatio_decile'] * decile_options['tov'].values[0])
                                    )

player_df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Player,Team,Age,OFFRTG,DEFRTG,NETRTG,AST%,...,wingspan_decile,2ptContested%_decile,3pt_CatchShoot_decile,3pt_Defense%_decile,DFG%_decile,Reb%_decile,AstRatio_decile,TovRatio_decile,season_position,decile_option 1 Total
0,0,0,0,Dwight Powell,DAL,30,113.2,110.6,2.5,8.1,...,3.0,9.0,7.0,4.0,4.0,3.0,7.0,6.0,2021_PF,1.85
1,5,5,5,Chris Boucher,TOR,29,111.0,105.7,5.3,2.2,...,8.0,7.0,2.0,1.0,0.0,7.0,0.0,0.0,2021_PF,2.31
2,15,15,15,Grant Williams,BOS,23,112.5,108.0,4.5,5.8,...,0.0,4.0,9.0,7.0,7.0,0.0,4.0,6.0,2021_PF,-0.39
3,18,18,18,Naz Reid,MIN,22,105.5,107.9,-2.4,9.1,...,7.0,4.0,6.0,8.0,0.0,3.0,2.0,8.0,2021_PF,0.21
4,20,20,20,Georges Niang,PHI,28,110.9,109.0,1.9,9.0,...,0.0,5.0,9.0,9.0,9.0,0.0,5.0,2.0,2021_PF,-0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515,2151,2151,2151,DeMarcus Cousins,SAC,23,105.4,106.0,-0.7,17.5,...,9.0,3.0,0.0,6.0,0.0,9.0,6.0,9.0,2013_C,0.54
2516,2209,2209,2209,Byron Mullens,PHI,25,95.7,112.2,-16.4,6.0,...,3.0,9.0,3.0,3.0,9.0,0.0,0.0,6.0,2013_C,-0.57
2517,2225,2225,2225,Meyers Leonard,POR,22,96.2,108.1,-12.0,8.0,...,6.0,6.0,0.0,0.0,3.0,6.0,9.0,0.0,2013_C,2.49
2518,2377,2377,2377,Mike Muscala,ATL,22,92.8,114.5,-21.7,5.8,...,0.0,0.0,0.0,9.0,6.0,3.0,3.0,3.0,2013_C,-1.86


In [51]:
decile_options = decile_options.rename(columns= {'Unnamed: 0': 'index'})
decile_options.head(3)

,index,height,wingspan,Contested2pt,CatchShoot3pt,Defense3pt,def_fg,reb,ast,tov,total
0,0,0.01,0.03,0.15,0.15,0.15,0.15,0.12,0.12,0.12,1
1,1,0.01,0.07,0.13,0.13,0.15,0.15,0.12,0.12,0.12,1
2,2,0.01,0.07,0.13,0.15,0.13,0.15,0.12,0.12,0.12,1


In [52]:
n = 0

for i in decile_options['index']:
    player_df['decile_option' + str(i) + ' Total'] =(
                                    (player_df['height_decile'] * decile_options['height'].values[i]) + 
                                    (player_df['wingspan_decile'] * decile_options['wingspan'].values[i]) +
                                    (player_df['2ptContested%_decile'] * decile_options['Contested2pt'].values[i]) +
                                    (player_df['3pt_CatchShoot_decile'] * decile_options['CatchShoot3pt'].values[i]) -
                                    (player_df['3pt_Defense%_decile'] * decile_options['Defense3pt'].values[i]) -
                                    (player_df['DFG%_decile'] * decile_options['def_fg'].values[i]) +
                                    (player_df['Reb%_decile'] * decile_options['reb'].values[i]) +
                                    (player_df['AstRatio_decile'] * decile_options['ast'].values[i]) -
                                    (player_df['TovRatio_decile'] * decile_options['tov'].values[i])
                                    )
    n += 1

player_df.head(3)

    


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Player,Team,Age,OFFRTG,DEFRTG,NETRTG,AST%,...,decile_option2840 Total,decile_option2841 Total,decile_option2842 Total,decile_option2843 Total,decile_option2844 Total,decile_option2845 Total,decile_option2846 Total,decile_option2847 Total,decile_option2848 Total,decile_option2849 Total
0,0,0,0,Dwight Powell,DAL,30,113.2,110.6,2.5,8.1,...,2.73,2.69,2.82,2.79,2.59,2.68,2.77,2.47,2.60,2.73
1,5,5,5,Chris Boucher,TOR,29,111.0,105.7,5.3,2.2,...,2.79,2.86,2.86,3.14,2.65,2.72,2.79,2.86,2.86,2.86
2,15,15,15,Grant Williams,BOS,23,112.5,108.0,4.5,5.8,...,-0.11,-0.15,-0.05,-0.11,-0.25,-0.19,-0.13,-0.37,-0.27,-0.17


## Add Team T-Score per Game

In [11]:
player_df = pd.read_excel('All_Player_Data_asofJuly8_deciles.xlsx')

In [12]:
data3 = pd.read_excel('Game_by_Game_Win_data_17-21.xlsx')

In [13]:
new_df = pd.merge(data3, minutes_df, on='Matchup_GameDate', how = 'left')
print(f' The shape of the DF: {new_df.shape}')
new_df.head(4)

 The shape of the DF: (11623, 2054)


,Unnamed: 0_x,Unnamed: 0.1,Unnamed: 0_x,Team,Match Up,Home_team_x,Away_team_x,GameDate,Hm_Date_id,W/L,...,HOU_Isaiah Canaan_min,NYK_Mindaugas Kuzminskas_min,MIL_Joel Bolomboy_min,BOS_Jarell Eddie_min,HOU_Brandan Wright_min,TOR_Nigel Hayes_min,MEM_Marquis Teague_min,POR_Georgios Papagiannis_min,HOU_Chinanu Onuaku_min,HOU_Aaron Jackson_min
0,0,0,1,CLE,MIL @ CLE,CLE,MIL,04/10/2022,CLE-04-10-2022,W,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,4,5,DAL,SAS @ DAL,DAL,SAS,04/10/2022,DAL-04-10-2022,W,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,6,7,POR,UTA @ POR,POR,UTA,04/10/2022,POR-04-10-2022,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,7,8,HOU,ATL @ HOU,HOU,ATL,04/10/2022,HOU-04-10-2022,L,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
game_df = new_df

In [56]:
game_df.head()

,Unnamed: 0,Unnamed: 0_x,Team,Match Up,Home_team_x,Away_team_x,GameDate,Hm_Date_id,W/L,MIN,...,HOU_Isaiah Canaan_min,NYK_Mindaugas Kuzminskas_min,MIL_Joel Bolomboy_min,BOS_Jarell Eddie_min,HOU_Brandan Wright_min,TOR_Nigel Hayes_min,MEM_Marquis Teague_min,POR_Georgios Papagiannis_min,HOU_Chinanu Onuaku_min,HOU_Aaron Jackson_min
0,0,1,CLE,MIL @ CLE,CLE,MIL,04/10/2022,CLE-04-10-2022,W,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,5,DAL,SAS @ DAL,DAL,SAS,04/10/2022,DAL-04-10-2022,W,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6,7,POR,UTA @ POR,POR,UTA,04/10/2022,POR-04-10-2022,L,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7,8,HOU,ATL @ HOU,HOU,ATL,04/10/2022,HOU-04-10-2022,L,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10,11,ORL,MIA @ ORL,ORL,MIA,04/10/2022,ORL-04-10-2022,W,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
df = game_df.columns[75:]
df

Index(['Matchup_GameDate', 'Unnamed: 0_y', 'MIL_Thanasis Antetokounmpo_min',
       'MIL_Jrue Holiday_min', 'MIL_Brook Lopez_min', 'MIL_Grayson Allen_min',
       'MIL_Jordan Nwora_min', 'MIL_Pat Connaughton_min',
       'MIL_Khris Middleton_min', 'MIL_George Hill_min',
       ...
       'HOU_Isaiah Canaan_min', 'NYK_Mindaugas Kuzminskas_min',
       'MIL_Joel Bolomboy_min', 'BOS_Jarell Eddie_min',
       'HOU_Brandan Wright_min', 'TOR_Nigel Hayes_min',
       'MEM_Marquis Teague_min', 'POR_Georgios Papagiannis_min',
       'HOU_Chinanu Onuaku_min', 'HOU_Aaron Jackson_min'],
      dtype='object', length=1979)

In [16]:
player_min_columns = list(game_df.columns[75:])

In [17]:
decile_options.shape

(2850, 12)

In [ ]:
for row in decile_options:
    df = game_df
    home_team = game_df['Home_team_x'].values[row]
    away_team = game_df['Away_team_x'].values[row]
    # New column for Decile Total 
    df['Home_Decile_Total_' + str(row)] = (
                                    home_players = df.loc[df[player_min_columns].astype(str).str[:3] == home_team]
                                    for player in home_players:
                                    (df[home_team + '_' + player] * decile_options['height'].values[row]) +
    )


In [77]:
# Define vlookup function TODO: This does not work
def vlookup(df1, df2, left_df_col, right_df_col, return_col):
    cols_keep = [right_df_col, return_col]
    df2 = df2[cols_keep]
    vlookup_df = pd.merge(df1,df2, left_on=left_df_col, right_on=right_df_col, how='left')
    return vlookup_df

In [ ]:
vlookup(game_df, player_df, 'MIL_George Hill_min', 'George Hill', 'Age' )

In [78]:
game_df.head(2)

,Unnamed: 0,Unnamed: 0_x,Team,Match Up,Home_team_x,Away_team_x,GameDate,Hm_Date_id,W/L,MIN,...,HOU_Isaiah Canaan_min,NYK_Mindaugas Kuzminskas_min,MIL_Joel Bolomboy_min,BOS_Jarell Eddie_min,HOU_Brandan Wright_min,TOR_Nigel Hayes_min,MEM_Marquis Teague_min,POR_Georgios Papagiannis_min,HOU_Chinanu Onuaku_min,HOU_Aaron Jackson_min
0,0,1,CLE,MIL @ CLE,CLE,MIL,04/10/2022,CLE-04-10-2022,W,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,5,DAL,SAS @ DAL,DAL,SAS,04/10/2022,DAL-04-10-2022,W,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
player_df.head(2)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Player,Team,Age,OFFRTG,DEFRTG,NETRTG,AST%,...,decile_option2840 Total,decile_option2841 Total,decile_option2842 Total,decile_option2843 Total,decile_option2844 Total,decile_option2845 Total,decile_option2846 Total,decile_option2847 Total,decile_option2848 Total,decile_option2849 Total
0,0,0,0,Dwight Powell,DAL,30,113.2,110.6,2.5,8.1,...,2.73,2.69,2.82,2.79,2.59,2.68,2.77,2.47,2.60,2.73
1,5,5,5,Chris Boucher,TOR,29,111.0,105.7,5.3,2.2,...,2.79,2.86,2.86,3.14,2.65,2.72,2.79,2.86,2.86,2.86


In [92]:
player_df.shape

(2520, 106)

## Problem in player_df:

- There are about 6000 player records in advanced_player_data2011-2021
- There are about 2500 player records in player_df
- Luka Doncic is not in 2020 player data? Why?

In [85]:
def get_Tscore(df, row, weight_option):
    row_data = df.iloc[row]
    home_columns = []
    away_columns = []

    # get home players
    home_team = row_data['Home_team_x']
    away_team = row_data['Away_team_x']
    for column in df.columns:
        if str(column)[0:3] == home_team:
            home_columns.append(column)
    home_minutes_df = row_data[home_columns]
    for column in df.columns:
        if str(column)[0:3] == away_team:
            away_columns.append(column)
    home_columns = pd.DataFrame(home_columns)
    home_columns_names = home_columns[0].values
    home_columns['home_players'] = home_columns[0].str[4:-4]

    #    Get home player minutes played
    home_columns['home_player_minutes'] = game_df[home_columns_names].values[1]
    # get away players
    away_columns = pd.DataFrame(away_columns)
    away_columns_names = away_columns[0].values
    away_columns['away_players'] = away_columns[0].str[4:-4]

    #    Get away player minutes played
    away_columns['away_player_minutes'] = game_df[away_columns_names].values[1]
    
    #   Add team t-scores for this weight option, and weigh by player minutes
    player_df['dec_option'] = 'decile_option'+ str(weight_option) + ' Total'
    keepsy = ['Player', 'season', 'dec_option', '3ptOpen%_decile', 'height_decile', 'wingspan_decile', '2ptContested%_decile', '3pt_CatchShoot_decile', '3pt_Defense%_decile', 'DFG%_decile', 'Reb%_decile', 'AstRatio_decile', 'TovRatio_decile']
    player_Tscores = player_df[keepsy]

     # Make sure correct Timing for T-score
    row_season = row_data['season']
    player_Tscores = player_Tscores.loc[player_Tscores['season'] == row_season - 1]
    home_columns = pd.merge(home_columns, player_Tscores, left_on='home_players', right_on ='Player', how='left')
       
    # get weights
    weights = decile_options.loc[decile_options['index'] == weight_option]
    home_columns['total_tscore'] = (
                                    (home_columns['height_decile'] * weights['height']) +
                                    (home_columns['wingspan_decile'] * weights['wingspan']) +
                                    (home_columns['2ptContested%_decile'] * weights['Contested2pt']) +
                                    (home_columns['3pt_CatchShoot_decile'] * weights['CatchShoot3pt']) -
                                    (home_columns['3pt_Defense%_decile'] * weights['Defense3pt']) +
                                    (home_columns['DFG%_decile'] * weights['def_fg']) +
                                    (home_columns['Reb%_decile'] * weights['reb']) +
                                    (home_columns['AstRatio_decile'] * weights['ast']) -
                                    (home_columns['TovRatio_decile'] * weights['tov'])
                                    )
    
    home_columns = home_columns.loc[home_columns['home_player_minutes'] > 0]
    
    return home_columns

In [91]:
test = get_Tscore(game_df, 1, 2)
test

,0,home_players,home_player_minutes,Player,season,dec_option,3ptOpen%_decile,height_decile,wingspan_decile,2ptContested%_decile,3pt_CatchShoot_decile,3pt_Defense%_decile,DFG%_decile,Reb%_decile,AstRatio_decile,TovRatio_decile,total_tscore
0,DAL_Luka Doncic_min,Luka Doncic,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DAL_Josh Green_min,Josh Green,17.0,Josh Green,2020.0,decile_option2 Total,0.0,5.0,6.0,7.0,0.0,7.0,8.0,7.0,6.0,8.0,NaN
2,DAL_Trey Burke_min,Trey Burke,9.0,Trey Burke,2020.0,decile_option2 Total,6.0,1.0,3.0,1.0,3.0,6.0,3.0,0.0,1.0,1.0,0.47
8,DAL_Boban Marjanovic_min,Boban Marjanovic,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,DAL_Dwight Powell_min,Dwight Powell,25.0,Dwight Powell,2020.0,decile_option2 Total,2.0,8.0,3.0,7.0,1.0,2.0,8.0,6.0,8.0,7.0,NaN
10,DAL_Dorian Finney-Smith_min,Dorian Finney-Smith,27.0,Dorian Finney-Smith,2020.0,decile_option2 Total,5.0,5.0,6.0,9.0,6.0,5.0,2.0,4.0,5.0,1.0,NaN
13,DAL_Reggie Bullock_min,Reggie Bullock,27.0,Reggie Bullock,2020.0,decile_option2 Total,8.0,8.0,4.0,4.0,8.0,0.0,7.0,2.0,2.0,1.0,NaN
14,DAL_Jalen Brunson_min,Jalen Brunson,25.0,Jalen Brunson,2020.0,decile_option2 Total,7.0,4.0,1.0,8.0,7.0,6.0,3.0,7.0,5.0,2.0,NaN
17,DAL_Marquese Chriss_min,Marquese Chriss,21.0,Marquese Chriss,2020.0,decile_option2 Total,1.0,6.0,3.0,2.0,1.0,9.0,1.0,9.0,2.0,5.0,NaN
18,DAL_Theo Pinson_min,Theo Pinson,13.0,Theo Pinson,2020.0,decile_option2 Total,0.0,6.0,8.0,9.0,0.0,9.0,9.0,4.0,0.0,5.0,NaN


In [67]:
weights = decile_options.loc[decile_options['index'] == 2]

In [73]:
weights['height']

2    0.01
Name: height, dtype: float64

In [130]:
decile_options.loc[decile_options['index'] == 0]

,index,height,wingspan,Contested2pt,CatchShoot3pt,Defense3pt,def_fg,reb,ast,tov,total
0,0,0.01,0.03,0.15,0.15,0.15,0.15,0.12,0.12,0.12,1


In [107]:
game_df.iloc[1]

Unnamed: 0                              4
Unnamed: 0_x                            5
Team                                  DAL
Match Up                        SAS @ DAL
Home_team_x                           DAL
                                  ...    
TOR_Nigel Hayes_min                     0
MEM_Marquis Teague_min                  0
POR_Georgios Papagiannis_min            0
HOU_Chinanu Onuaku_min                  0
HOU_Aaron Jackson_min                   0
Name: 1, Length: 2052, dtype: object